In [1]:
import pandas

In [5]:
df = pandas.read_csv('data/master_df.csv', index_col=0)

In [12]:
code_to_city = {}
code_to_state = {}
city_to_code = {}
for i,entry in df.dropna(subset=['code']).iterrows():
#     print(entry.field_office, entry.code)
    code_to_city[entry.code.strip()] = entry.field_office.strip()
    code_to_state[entry.code.strip()] = entry.state.strip()
    city_to_code[entry.field_office.strip()] = entry.code.strip()
    # discovered a few missing entries
code_to_city['CSC'] = float('nan')
code_to_state['CSC'] = 'California'

code_to_city['NBC'] = 'National Benefits Center'
code_to_state['NBC'] = float('nan')

code_to_city['NSC'] = 'NEBRASKA SERVICE CENTER'
code_to_state['NSC'] = 'Nebraska'

code_to_city['VSC'] = 'VERMONT SERVICE CENTER'
code_to_state['VSC'] = 'Vermont'

code_to_city['TSC'] = 'TEXAS SERVICE CENTER'
code_to_state['TSC'] = 'Texas'

# don't know what this is, and it doesn't have a label
code_to_city['GCU'] = "Garden City"
code_to_state['GCU'] = "New York"

code_to_city['ANC'], code_to_state['ANC']

('Anchorage', 'Alaska')

In [13]:
print(len(df))
df.sample(3)

2612


,field_office,Applications Received2,Approved3,Denied4,Pending5,quarter,state,code,year,fy
229,Charleston,659,549,45,"1,058","FY 2018, 4th Q, July 1-Sept. 30, 2018",South Carolina,CHS,2018.0,FY 2018
117,New Orleans,"1,012",839,92,"1,785","FY 2015, 4th Q, July 1-Sept. 30, 2015",Louisiana,NOL,2015.0,FY 2015
66,Detroit,740,7279,13,D,"FY 2019, 4th Q, July 1- Sept. 30, 2019",Michigan,DET,NaN,FY 2019


In [14]:
sum_df = df.dropna(subset=['code']).groupby(['code', 'fy']).sum()
sum_df

year
code fy             
1    FY 2016  2016.0
     FY 2017  8067.0
     FY 2018  8071.0
     FY 2019  4037.0
14   FY 2016  2016.0
...              ...
YAK  FY 2016  8063.0
     FY 2017  8067.0
     FY 2018  8071.0
     FY 2019  4037.0
     FY 2020     0.0

[719 rows x 1 columns]

In [15]:
sum_df['field_office'] = sum_df.index.map(lambda tup: code_to_city[tup[0]])

In [16]:
sum_df['state'] = sum_df.index.map(lambda tup: code_to_state[tup[0]])

In [17]:
sum_df['nationality'] = sum_df.state.map(
    lambda state: 'international' if state in ['United Kingdom', 'Italy', 'Germany', 'South Korea']
                  else 'US (territory)' if state in ['Guam', 'U.S. Virgin Islands', 'Puerto Rico']
                  else 'US'
)

In [18]:
sum_df[sum_df.nationality == 'US (territory)'].to_csv('data/sum_by_year_territory.csv')
sum_df[sum_df.nationality == 'US'].sort_values(by=['state', 'field_office']).to_csv('data/sum_by_year.csv')
sum_df[sum_df.nationality == 'international'].to_csv('data/sum_by_year_international.csv')